# 1 Preparación del entrono de desarrollo

## 1.1. Instalación de miniconda

Instalar miniconda: https://docs.anaconda.com/free/miniconda/index.html

## 1.2 Crear entorno zigbee

Entramos en miniconda a través del menú de inicio: Start > Anaconda Prompt (miniconda 3)

Desde la línea de comandos creamos el entrono python con nombre zigbee 

```
(base) ...> conda create -n zigbee python
```

Una vez creado el entorno, lo activamos e instalamos las librerías pyserial (manejo del puerto serie) y jupyter (entorno de desarrollo Jupyter lab)
```
(base) ...> conda activate zigbee
(zigbee) ...> conda install pyserial jupyter
```

## 1.3 Replicar el repositorio de la práctica

Desde el prompt de miniconda, nos movemos a la carpeta donde vamos a descargar el repositorio y lo clonamos

```
(base) ...> cd c:\src
(base) c:\src> git clone https://github.com/rpluma/zigbee.git
```

## 1.4 Uso del entorno de desarrollo

Entramos en miniconda a través del menú de inicio: Start > Anaconda Prompt (miniconda 3)
Cambiamos a la carpeta con el repositorio, activamos el entorno y ejecutamos jupyter lab

```
(base) ...> cd c:\src\zigbee
(base) c:\src\zigbee> conda activate zigbee
(zigbee) c:\src\zigbee> conda activate zigbee
(zigbee) c:\src\zigbee> jupyter lab
```


# 2 Desarrollo y pruebas

## 2.1 Detección de los puertos series en los que se han conectado los dispositivos

Desde Inicio -> Gestión de dispositivos -> Com Ports
- COM5: dispositivo 1
- COM6: dispositivo 2
- COM7: dispositivo 3
- COM8: dispositivo 4

## 2.2 Importación de librerías

In [1]:
import sys
import time
import serial
from math import exp # función exponencial

## 2.3 Función de conversión del valor de rssi (hexadecimal) a entero

In [2]:
def rssi2dist(rssi):
    x = int(rssi.strip(),16)
    y = 0.0536*exp(0.1215*x)
    return y
["{0}=>{1:.3f}".format(hex,rssi2dist(hex)) for hex in ['1B', '34', '40', '47']]


['1B=>1.425', '34=>29.719', '40=>127.714', '47=>298.955']

## 2.4 Definición de la clase Zigbee 

Empaqueta toda la funcionalidad requerida para un dispositivo:
- Método init: Constructor de la clase.
    - Inicializa el puerto serie con parámetros por defecto (9600 8-N-1)
    - El argumento dev es de la forma '/dev/ttyUSB*' o 'COM*'
- Método command_mode_start: Entra en modo comando con +++
- Método command_mode_exit: Sale del modo comando con ATCN
- Método command_at: envía un mensaje tipo AT concatenando un fin de línea e imprime el comando y la respuesta
- Método configurar: configura un dispositivo y guarda la configuración de forma permanente
- Método enviar: envía un mensaje cambiando el destino con ATDL cuando sea preciso

In [3]:
class Zigbee:
    puerto = None
    panid = None
    dlid = None
    
    def __init__(self, myid, device):
        self.myid = str(myid)
        self.puerto = serial.Serial(device, 9600, 
                            parity=serial.PARITY_NONE,
                            bytesize=serial.EIGHTBITS,
                            stopbits=serial.STOPBITS_ONE,
                            rtscts=False)
        
    def command_mode_start(self):
        time.sleep(1)   # guarda de 1 segundo
        self.puerto.write(b'+++') 
        time.sleep(1)   # ademas hay que esperar 1 segundo al OK
        print("{}>+++ ".format(self.myid, self.puerto.read(3).decode('utf-8')))
        return 

    def command_mode_exit(self):
        self.command_at('ATCN')
        return

    def command_at(self, msg): 
        time.sleep(1)
        toSerial = msg+"\r\n" # concatenar LF, CR
        self.puerto.write(toSerial.encode('ascii', 'replace'))
        self.puerto.flush()
        time.sleep(.2)
        bytesToRead = self.puerto.inWaiting() # imprimir respuesta
        fromSerial = self.puerto.read(bytesToRead)
        print ("{}>{}: [{}]".format(self.myid, msg, fromSerial.decode('utf-8').strip()))
        return fromSerial.decode('utf-8')
        
    def configurar(self, panid='3210'):
        self.panid = str(panid)             # guardar el valor de panid como string        
        self.command_mode_start()           # entrar en modo comando con +++
        self.command_at('AT')               # comprobar que se aceptan comandos AT
        self.command_at('ATRE')             # resetea la configuración guardada
        self.command_at('ATID '+self.panid) # definir el PANID
        self.command_at('ATDH 0')           # definir direcciones de 16 bits
        self.command_at('ATMY '+self.myid)  # definir la dirección MY addres
        self.command_at('ATWR')             # guardar la configuración de forma permanente
        # self.command_at('ATCH')
        self.command_mode_exit()            # salir del modo +++

    def enviar(self, destino='2', msg='mensaje'):
        if self.dlid != str(destino):
            self.dlid = str(destino)
            self.command_mode_start()         # entrar en modo comando con +++
            self.command_at('ATDL '+self.dlid)     # establecer el destino del mensaje        
            self.command_mode_exit()          # salir del modo +++
        self.puerto.write(msg.encode('ascii', 'replace'))
        self.puerto.flush()
        time.sleep(.1)
    
    def recibir(self, bGetDb=True):
        dist = None
        
        bytesToRead = self.puerto.inWaiting() # imprimir respuesta
        fromSerial = self.puerto.read(bytesToRead)
        if bGetDb:
            self.command_mode_start()      # entrar en modo comando con +++
            rssi = self.command_at('ATDB') # obtener el RSSI
            dist = rssi2dist(rssi);
            self.command_mode_exit()       # salir del modo +++            
            print("RSSI=[{}] => Distancia={:.2f} (m)".format(rssi.strip(), dist))
        return (fromSerial, dist)

    def cerrar(self):
        self.puerto.close()
        
        

## 2.5 Creación de objetos Zigbee para cada uno de los cuatro dispositivos

In [4]:
z1 = Zigbee(1, "COM5")
z2 = Zigbee(2, "COM6")
z3 = Zigbee(3, "COM8")
z4 = Zigbee(4, "COM9")

SerialException: could not open port 'COM5': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

## 2.6 Configuración de cada objeto

### Configuración del objeto z1

Sólo es necesario la primera vez, ya que la configuración se guarda con ATWR

In [5]:
z1.configurar()

1>+++ 
1>AT: [OK]
1>ATRE: [OK]
1>ATID 3210: [OK]
1>ATDH 0: [OK]
1>ATMY 1: [OK]
1>ATWR: [OK]
1>ATCN: [OK]


### Configuración de z2, z3 y z4

In [6]:
z2.configurar()
z3.configurar()
z4.configurar()

2>+++ 
2>AT: [OK]
2>ATRE: [OK]
2>ATID 3210: [OK]
2>ATDH 0: [OK]
2>ATMY 2: [OK]
2>ATWR: [OK]
2>ATCN: [OK]
3>+++ 
3>AT: [OK]
3>ATRE: [OK]
3>ATID 3210: [OK]
3>ATDH 0: [OK]
3>ATMY 3: [OK]
3>ATWR: [OK]
3>ATCN: [OK]
4>+++ 
4>AT: [OK]
4>ATRE: [OK]
4>ATID 3210: [OK]
4>ATDH 0: [OK]
4>ATMY 4: [OK]
4>ATWR: [OK]
4>ATCN: [OK]


## 2.7 Envío de datos para obtener la distancia

### Envío de datos desde z2 a z1

La primera vez que se invoca a enviar cambia la dirección de destino así que hay que utilizar ATDL

In [7]:
z2.enviar('1', " mensaje 1 de 2 a 1")

2>+++ 
2>ATDL 1: [OK]
2>ATCN: [OK]


Los siguientes envíos ya no hace falta cambiar el destino y el objeto no utiliza ATDL

In [8]:
z2.enviar('1', " mensaje 2 de 2 a 1")

La recepción en 1 indica la distancia de z2 a z1

In [9]:
resp, d2 = z1.recibir()
print("Respuesta=[{}], distancia={:.3f}".format(resp.strip(), d2))

1>+++ 
1>ATDB: [37]
1>ATCN: [OK]
RSSI=[37] => Distancia=42.79 (m)
Respuesta=[b'mensaje 1 de 2 a 1 mensaje 2 de 2 a 1'], distancia=42.790


In [10]:
z3.enviar('1', " mensaje 1 de 3 a 1")
resp3, d3 = z1.recibir()

3>+++ 
3>ATDL 1: [OK]
3>ATCN: [OK]
1>+++ 
1>ATDB: [3C]
1>ATCN: [OK]
RSSI=[3C] => Distancia=78.55 (m)


In [11]:
z4.enviar('4', " mensaje 1 de 4 a 1")
resp4, d4 = z1.recibir()

4>+++ 
4>ATDL 4: [OK]
4>ATCN: [OK]
1>+++ 
1>ATDB: [0]
1>ATCN: [OK]
RSSI=[0] => Distancia=0.05 (m)


In [12]:
"Distancias: d2={:.3f}, d3={:.3f}, d4={:.3f}".format(d2, d3, d4)

'Distancias: d2=42.790, d3=78.555, d4=0.054'

In [13]:
z1.cerrar()
z2.cerrar()
z3.cerrar()
z4.cerrar()